## Imports

In [1]:
import random
import altair as alt
import pandas as pd
import numpy as np
from sklearn import set_config
from sklearn.compose import make_column_transformer
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import Ridge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import RandomizedSearchCV, cross_validate
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

np.random.seed(123) # setting the seed for repoducibility

**Dataset source**: https://github.com/GMU-CherryBlossomCompetition/peak-bloom-prediction/tree/main/data

## 1. Initial Observation<a name="1"></a>

In [2]:
url_kyoto = "https://raw.githubusercontent.com/SotongMarkotong/Cherry-Blossom-Predictor/main/kyoto.csv"
kyoto_df = pd.read_csv(url_kyoto)
kyoto_df.head()

,location,lat,long,alt,year,bloom_date,bloom_doy
0,kyoto,35.011983,135.676114,44,812,0812-04-01,92
1,kyoto,35.011983,135.676114,44,815,0815-04-15,105
2,kyoto,35.011983,135.676114,44,831,0831-04-06,96
3,kyoto,35.011983,135.676114,44,851,0851-04-18,108
4,kyoto,35.011983,135.676114,44,853,0853-04-14,104


**Features:**
1. location: The location of the cherry blossom. All values are the same, which is in Kyoto.
2. lat: The latitude of the observation.
3. long: The longitude of the observation.
4. alt: The altitude of the observation.
5. year: The year of the bloom observation.
6. bloom_date: The date of peak bloom of the cherry trees.
7. bloom_doy: Number of days since January 1st until peak bloom. (January 1st = 1)
<br>
<br>

## 2. Data splitting <a name="2"></a>

In [3]:
kyoto_train, kyoto_test = train_test_split(kyoto_df, test_size = 0.25, random_state = 123)
kyoto_train.head()

,location,lat,long,alt,year,bloom_date,bloom_doy
200,kyoto,35.011983,135.676114,44,1316,1316-04-12,103
625,kyoto,35.011983,135.676114,44,1809,1809-04-17,107
599,kyoto,35.011983,135.676114,44,1783,1783-04-13,103
703,kyoto,35.011983,135.676114,44,1888,1888-04-16,107
369,kyoto,35.011983,135.676114,44,1533,1533-04-21,111


Splitting the dataset into training and testing with size 0.75 and 0.25 respectively. This splitting is to make sure that the model is not influenced by the test set at all. random_state is used for replicability.<br>
<br>

## 3. Exploratory Data Analysis<a name="3"></a>

In [4]:
kyoto_summary = kyoto_train.describe(include = "all")
kyoto_summary

,location,lat,long,alt,year,bloom_date,bloom_doy
count,626,6.260000e+02,6.260000e+02,626.0,626.000000,626,626.000000
unique,1,NaN,NaN,NaN,NaN,626,NaN
top,kyoto,NaN,NaN,NaN,NaN,1316-04-12,NaN
freq,626,NaN,NaN,NaN,NaN,1,NaN
mean,NaN,3.501198e+01,1.356761e+02,44.0,1555.563898,NaN,104.397764
std,NaN,4.977777e-14,1.052444e-12,0.0,304.002947,NaN,6.542166
min,NaN,3.501198e+01,1.356761e+02,44.0,812.000000,NaN,84.000000
25%,NaN,3.501198e+01,1.356761e+02,44.0,1334.500000,NaN,100.000000
50%,NaN,3.501198e+01,1.356761e+02,44.0,1588.500000,NaN,104.500000
75%,NaN,3.501198e+01,1.356761e+02,44.0,1805.750000,NaN,109.000000


In [5]:
kyoto_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 626 entries, 200 to 510
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   location    626 non-null    object 
 1   lat         626 non-null    float64
 2   long        626 non-null    float64
 3   alt         626 non-null    int64  
 4   year        626 non-null    int64  
 5   bloom_date  626 non-null    object 
 6   bloom_doy   626 non-null    int64  
dtypes: float64(2), int64(3), object(2)
memory usage: 39.1+ KB


**Observations from summary statistics:**
We can see that the count from describe() is the same as the Non-Null count from info() for all features. This shows that there are no features with missing values, and we do not need to do imputation. We can also see that some features are numeric and mostly will need scaling since the range = (min to max) of each feature is different. The data seems also to be already tidied so no further processing other than scaling, dropping, or encoding should be done.<br>
<br>
<br>

## 4. Preprocessing and transformations <a name="4"></a>
<hr>

Now we split the features and group them with their characteristics. We decided to drop the location feature as all we're only considering Kyoto's bloom day and all locations are the same, hence there is no predictive value from location feature. We're also dropping bloom_date as it is the same representation with bloom_doy in date format. All other features are numeric and will be scaled with standardization.

In [6]:
numeric_features = ['lat', 'long', 'alt', 'year']
drop_features = ["location", "bloom_date"]
target = "bloom_doy"
print("numeric features:", numeric_features)
print("drop features:", drop_features)
print("target:", target)

numeric features: ['lat', 'long', 'alt', 'year']
drop features: ['location', 'bloom_date']
target: bloom_doy


**So**, there are 3 feature types in our case: numeric features, drop features, and the target. We will create a column_transformer consisting of both a numeric transformer(Scaling) and a drop_transformer(drop).
 <br>

In [7]:
numeric_transformer = StandardScaler()

preprocessor = make_column_transformer(
    (numeric_transformer, numeric_features),
    ("drop", drop_features)
)
preprocessor

ColumnTransformer(transformers=[('standardscaler', StandardScaler(),
                                 ['lat', 'long', 'alt', 'year']),
                                ('drop', 'drop', ['location', 'bloom_date'])])

Now, we should split the train and test data sets into the predictors(X) and target(Y).

In [8]:
X_train = kyoto_train.drop(columns = target)
y_train = kyoto_train[target]
X_test = kyoto_test.drop(columns = target)
y_test = kyoto_test[target]
X_train.head()

,location,lat,long,alt,year,bloom_date
200,kyoto,35.011983,135.676114,44,1316,1316-04-12
625,kyoto,35.011983,135.676114,44,1809,1809-04-17
599,kyoto,35.011983,135.676114,44,1783,1783-04-13
703,kyoto,35.011983,135.676114,44,1888,1888-04-16
369,kyoto,35.011983,135.676114,44,1533,1533-04-21


## 5. Baseline model <a name="5"></a>
<hr>

We will create a baseline model to provide us a reference point as the minimum acceptable performance for our model. Since it is a regression problem, we will use DummyRegressor as the baseline with median as it's strategy. But before we start, we will create a function mean_std_cross_val_scores() that returns a panda series containing cross validation(5-fold) results including the average fit and score time, and more importantly the average test and train score with its standard deviation. The scoring of our models will use the root mean squared error strategy (the smaller the better).

In [9]:
def mean_std_cross_val_scores(model, X_train, y_train, **kwargs):
    """
    Returns mean and std of cross validation

    Parameters
    ----------
    model :
        scikit-learn model
    X_train : numpy array or pandas DataFrame
        X in the training data
    y_train :
        y in the training data

    Returns
    ----------
        pandas Series with mean scores from cross_validation
    """

    scores = cross_validate(model, X_train, y_train, **kwargs)

    mean_scores = pd.DataFrame(scores).mean()
    std_scores = pd.DataFrame(scores).std()
    out_col = []

    for i in range(len(mean_scores)):
        if i > 1:
            out_col.append((f"%0.3f (+/- %0.3f)" % (mean_scores[i]*-1, std_scores[i]*-1)))
        else:
            out_col.append((f"%0.3f (+/- %0.3f)" % (mean_scores[i], std_scores[i])))

    return pd.Series(data=out_col, index=mean_scores.index)

In [10]:
results_dict = {}
dummy = DummyRegressor(strategy = "median")
dummy_pipe = make_pipeline(preprocessor, dummy)
dummy_pipe.fit(X_train, y_train)
results_dict["dummy"] = mean_std_cross_val_scores(
    dummy_pipe, X_train, y_train, cv=5, scoring = "neg_root_mean_squared_error", return_train_score=True
)
results_dict = pd.DataFrame(results_dict).T
results_dict

/tmp/ipykernel_245/1816060613.py:29: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  out_col.append((f"%0.3f (+/- %0.3f)" % (mean_scores[i], std_scores[i])))
/tmp/ipykernel_245/1816060613.py:27: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  out_col.append((f"%0.3f (+/- %0.3f)" % (mean_scores[i]*-1, std_scores[i]*-1)))


,fit_time,score_time,test_score,train_score
dummy,0.005 (+/- 0.000),0.002 (+/- 0.000),6.565 (+/- -0.565),6.547 (+/- -0.142)


Hence, our baseline model has a validation score of 6.565 and a train score of 6.547.<br>
<br>

## 6. Linear model(Ridge) <a name="6"></a>
<hr>

We will use a linear model for this regression problem, and we choose ridge as our model since it has a hyperparameter that controls complexity which can be optimized later on. First we will use the default value to check its performance.

In [11]:
ridge_default = {}

## Ridge regression model with default hyperparameter alpha value.
ridge = Ridge(random_state = 123, alpha = 1)
ridge_pipe = make_pipeline(preprocessor, ridge)
ridge_pipe.fit(X_train, y_train)

ridge_default["alpha = 1.0"] = mean_std_cross_val_scores(
    ridge_pipe, X_train, y_train, cv = 5, scoring = "neg_root_mean_squared_error", return_train_score=True
)
ridge_default = pd.DataFrame(ridge_default).T
ridge_default

/tmp/ipykernel_245/1816060613.py:29: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  out_col.append((f"%0.3f (+/- %0.3f)" % (mean_scores[i], std_scores[i])))
/tmp/ipykernel_245/1816060613.py:27: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  out_col.append((f"%0.3f (+/- %0.3f)" % (mean_scores[i]*-1, std_scores[i]*-1)))


,fit_time,score_time,test_score,train_score
alpha = 1.0,0.006 (+/- 0.001),0.002 (+/- 0.000),6.516 (+/- -0.536),6.505 (+/- -0.131)


Hence, our ridge model has a validation score of 6.516 and a train score of 6.505. Let's try to improve this score with hyperaparemeter optimization. <br>
<br>

**Hyperparameter Optimization Alpha:**

In [12]:
train_scores = []
test_scores = []

alpha_vals = 10.0 ** np.arange(-2, 2.5, 0.5)

for A in alpha_vals:
    test_pipe = make_pipeline(preprocessor, Ridge(alpha = A))    
    cv_results = mean_std_cross_val_scores(test_pipe, X_train, y_train, scoring = "neg_root_mean_squared_error", return_train_score=True)
    train_scores.append(cv_results["train_score"])
    test_scores.append(cv_results["test_score"])
    
pd.DataFrame({"alpha": alpha_vals, "test_score": test_scores, "train_score": train_scores})

/tmp/ipykernel_245/1816060613.py:29: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  out_col.append((f"%0.3f (+/- %0.3f)" % (mean_scores[i], std_scores[i])))
/tmp/ipykernel_245/1816060613.py:27: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  out_col.append((f"%0.3f (+/- %0.3f)" % (mean_scores[i]*-1, std_scores[i]*-1)))
/tmp/ipykernel_245/1816060613.py:29: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  out_col.append((f"%0.3f (+/- %0.3f)

,alpha,test_score,train_score
0,0.010000,6.516 (+/- -0.536),6.505 (+/- -0.131)
1,0.031623,6.516 (+/- -0.536),6.505 (+/- -0.131)
2,0.100000,6.516 (+/- -0.536),6.505 (+/- -0.131)
3,0.316228,6.516 (+/- -0.536),6.505 (+/- -0.131)
4,1.000000,6.516 (+/- -0.536),6.505 (+/- -0.131)
5,3.162278,6.516 (+/- -0.536),6.505 (+/- -0.131)
6,10.000000,6.516 (+/- -0.537),6.505 (+/- -0.131)
7,31.622777,6.515 (+/- -0.539),6.505 (+/- -0.131)
8,100.000000,6.514 (+/- -0.543),6.505 (+/- -0.131)


Here we can see that for different values of hyperparameter Alpha, the test and train score relatively stays the same. In this case, we would jsut pick the default value as complexcity does not matter for Ridge model with this dataset.<br>
<br>

## 7. Performance on the test set <a name="7"></a>
<hr>

In [13]:
best_model = Ridge(alpha = 1.0)
best_pipe = make_pipeline(preprocessor, best_model)
best_pipe.fit(X_train, y_train)

y_pred = pd.Series(best_pipe.predict(X_test))

test_score = mean_squared_error(
    y_true = y_test,
    y_pred = y_pred
)**(1/2)

print("Test Score:", test_score)

Test Score: 6.472293648117765


Hence the test score of our regression model is approximately 6.47. which is very similar with our validation score. This leads to our belief of no optimization bias occuring in our model.

## 8. Conclusion and Discussion <a name="8"></a>
<hr>

The Ridge model's score did not differ much from our baseline model's score, this shows that either our model does not generalize very well for this dataset, or the baseline model already gave a decent generalization for this dataset. The problem seems to be more possible for the former. We could try to use different regression models for the future, but I believe a classification model would work good too. An example of classification model that could be used are Boosted Trees(XgBoost), K-NN, or even logistic regression. 